<h1>Enrichment and transformations<h1>

In [1]:
import pandas as pd
import numpy as np
import requests

<h2>read the JSON file that you saved in ex02<h2>

In [2]:
df = pd.read_json("../../data/auto.json")
pd.options.display.float_format = '{:.2f}'.format
df["Fines"]


0      3200.00
1      6500.00
2      2100.00
3      2000.00
4      6100.00
        ...   
720   11400.00
721    1600.00
722   22300.00
723     600.00
724    8594.59
Name: Fines, Length: 725, dtype: float64

<h2>enrich the dataframe using a sample from that dataframe<h2>

In [3]:
np.random.seed(21)
sample_df = pd.DataFrame()
sample_df[["CarNumber", "Make", "Model"]] = df.sample(random_state=21, n = 200)[["CarNumber", "Make", "Model"]]
sample_df["Fines"] = np.random.choice(df["Fines"], size=200)
sample_df["Refund"] = np.random.choice(df["Refund"], size=200)
concat_rows = pd.concat([df, sample_df], ignore_index=True)


<h2>enrich the dataframe concat_rows by a new column with the data generated<h2>

In [4]:
np.random.seed(21)
Year = pd.Series(np.random.randint(1980, 2019, size=concat_rows.index.size))
fines = pd.concat([concat_rows, Year], axis=1)
fines.rename({0:"Year"}, inplace=True, axis=1)

<h2>enrich the dataframe with the data from another dataframe<h2>

In [5]:
most_popular_surnames = pd.read_json("../../datasets/surname.json", orient="values")
most_popular_surnames.columns = most_popular_surnames.iloc[0]


In [6]:
surnames = most_popular_surnames["NAME"].sample(n = fines["CarNumber"].nunique(), replace=True, random_state=21)

In [7]:
owners = pd.DataFrame()
owners["CarNumber"] = fines["CarNumber"].unique()
owners = pd.concat([owners, surnames.reset_index(drop=True)], axis=1, ignore_index=True)
owners.rename({0:"CarNumber", 1:"SURNAME"}, axis=1, inplace=True)

In [8]:
new_numbers = []
fines = pd.concat([fines, pd.DataFrame({
    'CarNumber': ['X452P8912RUS', 'M739K4567RUS', 'L821H3478RUS', 'N634T5678RUS', 'Q912S2345RUS'],
    'Refund': [1, 3, 2, 4, 2],
    'Fines': [2800.0, 4200.0, 3500.0, 3800.0, 2950.0],
    'Make': ['Toyota', 'Chevrolet', 'Honda', 'Nissan', 'Volkswagen'],
    'Model': ['Corolla', 'Camaro', 'Civic', 'Sentra', 'Golf'],
    'Year': [1988, 1990, 1987, 1991, 1989]
})], ignore_index=True)

In [ ]:
owners = owners.iloc[:-20]
owners = pd.concat([owners, pd.DataFrame({
    'CarNumber': ['Y163O8161RUS', 'A456B7890XYZ', 'C123D4567EFG'],
    'SURNAME': ['REYES', 'SMITH', 'JOHNSON']
})], ignore_index=True)

,CarNumber,SURNAME
0,Y163O8161RUS,REYES
1,E432XX77RUS,ROGERS
2,7184TT36RUS,MORALES
3,X582HE161RUS,ANDERSON
4,E34877152RUS,LONG
...,...,...
509,7360C8197RUS,LEE
510,E29677161RUS,HARRIS
511,Y163O8161RUS,REYES
512,A456B7890XYZ,SMITH


In [11]:
pd.merge(left=fines, right=owners, on = "CarNumber", how="inner")

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,REYES
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROGERS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORALES
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,ANDERSON
4,E34877152RUS,2,6100.00,Ford,Focus,2014,LONG
...,...,...,...,...,...,...,...
888,X786CO96RUS,1,3000.00,Ford,Focus,1996,YOUNG
889,9764HY161RUS,1,8594.59,Ford,Focus,2002,BAKER
890,H906YK197RUS,1,100.00,Ford,Focus,1996,COX
891,K2797K154RUS,1,8800.00,Ford,Focus,2012,SANDERS


In [12]:
pd.merge(left=fines, right=owners, on = "CarNumber", how="outer")

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,704687163RUS,2,1400.00,Ford,Focus,2010,JACKSON
1,704687163RUS,2,8000.00,Ford,Focus,2005,JACKSON
2,704787163RUS,2,2800.00,Ford,Focus,2002,ANDERSON
3,704987163RUS,2,8594.59,Ford,Focus,2014,EVANS
4,705287163RUS,2,2000.00,Ford,Focus,1985,SANDERS
...,...,...,...,...,...,...,...
925,Y969O8197RUS,2,7800.00,Ford,Focus,2007,SCOTT
926,Y973O8197RUS,2,8594.59,Ford,Focus,1981,FLORES
927,Y973O8197RUS,1,34800.00,Ford,Focus,1998,FLORES
928,Y973O8197RUS,1,69600.00,Ford,Focus,1991,FLORES


In [13]:
pd.merge(left=fines, right=owners, on = "CarNumber", how="left")

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,REYES
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROGERS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORALES
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,ANDERSON
4,E34877152RUS,2,6100.00,Ford,Focus,2014,LONG
...,...,...,...,...,...,...,...
925,X452P8912RUS,1,2800.00,Toyota,Corolla,1988,NaN
926,M739K4567RUS,3,4200.00,Chevrolet,Camaro,1990,NaN
927,L821H3478RUS,2,3500.00,Honda,Civic,1987,NaN
928,N634T5678RUS,4,3800.00,Nissan,Sentra,1991,NaN


In [14]:
pd.merge(left=fines, right=owners, on = "CarNumber", how="right")

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,REYES
1,Y163O8161RUS,2,1600.00,Ford,Focus,2018,REYES
2,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROGERS
3,E432XX77RUS,2,13000.00,Toyota,Camry,2004,ROGERS
4,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORALES
...,...,...,...,...,...,...,...
888,O421OT161RUS,1,1200.00,Skoda,Octavia,1987,PRICE
889,O421OT161RUS,2,900.00,Skoda,Octavia,1992,PRICE
890,7360C8197RUS,2,9600.00,Ford,Focus,1989,LEE
891,7360C8197RUS,2,600.00,Ford,Focus,1999,LEE


<h2>create a pivot table from the fines dataframe, it should look like this (the values are the sums of the fines), but with all the years (the values may be different for you):<h2>

In [20]:
fines.pivot_table(columns="Year", index=["Make", "Model"], values="Fines", aggfunc="sum")

Year                    1980      1981      1982     1983     1984      1985  \
Make       Model                                                               
Chevrolet  Camaro        NaN       NaN       NaN      NaN      NaN       NaN   
Ford       Focus   120783.76 233289.17 154894.59 57889.17 55789.17  91483.76   
           Mondeo        NaN       NaN       NaN      NaN      NaN       NaN   
Honda      Civic         NaN       NaN       NaN      NaN      NaN       NaN   
Nissan     Sentra        NaN       NaN       NaN      NaN      NaN       NaN   
Skoda      Octavia   3100.00  83000.00   8594.59 10600.00      NaN 151500.00   
Toyota     Camry     8594.59   4400.00  22600.00  7900.00      NaN       NaN   
           Corolla       NaN       NaN       NaN      NaN      NaN       NaN   
Volkswagen Golf       800.00   9300.00       NaN 24000.00  5800.00   3800.00   
           Jetta         NaN       NaN   4000.00      NaN  1500.00       NaN   
           Passat        NaN       NaN       NaN      NaN  1600.00  31700.00   
           Touareg       NaN       NaN       NaN      NaN      NaN       NaN   

Year                    1986      1987     1988     1989  ...      2009  \
Make       Model                                          ...             
Chevrolet  Camaro        NaN       NaN      NaN      NaN  ...       NaN   
Ford       Focus   202178.35 299200.00 71089.17 94800.00  ... 297500.00   
           Mondeo        NaN       NaN      NaN      NaN  ...       NaN   
Honda      Civic         NaN   3500.00      NaN      NaN  ...       NaN   
Nissan     Sentra        NaN       NaN      NaN      NaN  ...       NaN   
Skoda      Octavia       NaN   1200.00 10100.00  1600.00  ...  17094.59   
Toyota     Camry         NaN       NaN      NaN 22400.00  ...       NaN   
           Corolla       NaN   3200.00 10400.00      NaN  ...  16594.59   
Volkswagen Golf      8594.59   1500.00      NaN  2950.00  ...    500.00   
           Jetta         NaN       NaN      NaN      NaN  ...       NaN   
           Passat    8594.59   1600.00      NaN 13000.00  ...       NaN   
           Touareg       NaN       NaN      NaN      NaN  ...       NaN   

Year                   2010      2011      2012      2013      2014      2015  \
Make       Model                                                                
Chevrolet  Camaro       NaN       NaN       NaN       NaN       NaN       NaN   
Ford       Focus   66789.17 188183.76 127500.00 291789.17 151078.35 178583.76   
           Mondeo       NaN       NaN       NaN       NaN       NaN       NaN   
Honda      Civic        NaN       NaN       NaN       NaN       NaN       NaN   
Nissan     Sentra       NaN       NaN       NaN       NaN       NaN       NaN   
Skoda      Octavia  8400.00       NaN    900.00  10100.00  11594.59       NaN   
Toyota     Camry    7200.00  19800.00       NaN       NaN       NaN       NaN   
           Corolla      NaN   4000.00  23800.00       NaN       NaN   9600.00   
Volkswagen Golf         NaN       NaN    300.00  20800.00       NaN  18400.00   
           Jetta    5700.00       NaN       NaN   9000.00       NaN       NaN   
           Passat  10000.00   9500.00   3000.00  15000.00       NaN       NaN   
           Touareg      NaN       NaN   6300.00   8594.59       NaN   1300.00   

Year                    2016      2017      2018  
Make       Model                                  
Chevrolet  Camaro        NaN       NaN       NaN  
Ford       Focus   188589.17  80778.35 125589.17  
           Mondeo        NaN       NaN       NaN  
Honda      Civic         NaN       NaN       NaN  
Nissan     Sentra        NaN       NaN       NaN  
Skoda      Octavia  11594.59    500.00   8600.00  
Toyota     Camry     8594.59   1000.00       NaN  
           Corolla       NaN       NaN  34300.00  
Volkswagen Golf       400.00 175300.00   1000.00  
           Jetta         NaN       NaN       NaN  
           Passat        NaN    600.00       NaN  
           Touareg       NaN  

<h2>save both the fines and owners dataframes to CSV files without an index<h2>

In [21]:
fines.to_csv("../../data/fines.csv", index=False)
owners.to_csv("../../data/owners.csv", index=False)